In [2]:
from pydantic import BaseModel

# Basic Model

In [3]:
class Person(BaseModel):
    first_name: str
    last_name: str
    age: int

In [5]:
p1 = Person(first_name="Jemima", last_name="Briones Huerta", age=2)
p1

Person(first_name='Jemima', last_name='Briones Huerta', age=2)

In [6]:
from pydantic import ValidationError

In [9]:
try:
    p2 = Person(first_name="Catherine", last_name="Briones Huerta", age="1")
except ValidationError as err:
    print(err)

In [10]:
p1.first_name

'Jemima'

In [11]:
p2.first_name

'Catherine'

In [12]:
p1 = Person(first_name="Michelina", last_name="Briones Huerta", age=13)

In [13]:
p1.first_name

'Michelina'

In [16]:
# Watch out, arbitrary assignment happens
p1.age = "chill"
p1.age

'chill'

# Validation Exceptions

In [23]:
try:
    p2 = Person(first_name="Catherine", last_name=22, age="hello there")

except ValidationError as err:
    print(err.json())
    

[{"type":"string_type","loc":["last_name"],"msg":"Input should be a valid string","input":22,"url":"https://errors.pydantic.dev/2.9/v/string_type"},{"type":"int_parsing","loc":["age"],"msg":"Input should be a valid integer, unable to parse string as an integer","input":"hello there","url":"https://errors.pydantic.dev/2.9/v/int_parsing"}]


In [20]:
err.json()

NameError: name 'err' is not defined

# Deserializing Data

In [28]:
data = {
    "first_name": "Jemima",
    "last_name": "Briones Huerta",
    "age": 2
}

data_json = """
{
    "first_name": "Jemima",
    "last_name": "Briones Huerta",
    "age": 2
}
"""

In [30]:
p = Person.model_validate(data)
p
p2 = Person.model_validate_json(data_json)
p2

Person(first_name='Jemima', last_name='Briones Huerta', age=2)

# Required vs Optional Fields

In [31]:
try:
    Person(age=10)
except ValidationError as err:
    print(err)

2 validation errors for Person
first_name
  Field required [type=missing, input_value={'age': 10}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.9/v/missing
last_name
  Field required [type=missing, input_value={'age': 10}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.9/v/missing


In [32]:
data = {"age": 10}
try:
    Person.model_validate(data)
except ValidationError as err:
    print(err)

2 validation errors for Person
first_name
  Field required [type=missing, input_value={'age': 10}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.9/v/missing
last_name
  Field required [type=missing, input_value={'age': 10}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.9/v/missing


In [33]:
class Person(BaseModel):
    first_name: str
    last_name: str
    age: int = 0

In [34]:
Person.model_fields

{'first_name': FieldInfo(annotation=str, required=True),
 'last_name': FieldInfo(annotation=str, required=True),
 'age': FieldInfo(annotation=int, required=False, default=0)}

In [35]:
p = Person(first_name="Jemima", last_name="Briones")
p

Person(first_name='Jemima', last_name='Briones', age=0)

# Nullable Fields

In [39]:
class Person(BaseModel):
    first_name: str
    last_name: str | None = None  # or Union[str, None]
    age: int = 0

In [38]:
Person.model_fields

{'first_name': FieldInfo(annotation=str, required=True),
 'last_name': FieldInfo(annotation=Union[str, NoneType], required=False, default=None),
 'age': FieldInfo(annotation=int, required=False, default=0)}

In [41]:
from typing import List

In [44]:
class Person(BaseModel):
    first_name: str
    last_name: str | None = None
    age: int = 0
    lucky_numbers: list[int] = []

In [45]:
Person.model_fields

{'first_name': FieldInfo(annotation=str, required=True),
 'last_name': FieldInfo(annotation=Union[str, NoneType], required=False, default=None),
 'age': FieldInfo(annotation=int, required=False, default=0),
 'lucky_numbers': FieldInfo(annotation=list[int], required=False, default=[])}

In [48]:
p = Person(first_name="Charbel", lucky_numbers=[1, "2", 3.0])  # Coersion to ints
p

Person(first_name='Charbel', last_name=None, age=0, lucky_numbers=[1, 2, 3])

In [49]:
[type(n) for n in p.lucky_numbers]

[int, int, int]

# Aliases and the Field Class

In [50]:
from pydantic import Field

data = {
    "id": 100,
    "First Name": "Jemima",
    "LASTNAME": "Briones",
    "age in years": 3
}

In [51]:
class Person(BaseModel):
    id_: int = Field(alias="id")
    first_name: str = Field(alias="First Name")
    last_name: str = Field(alias="LASTNAME")
    age: int = Field(alias="age in years")
    

In [52]:
p = Person.model_validate(data)
p

Person(id_=100, first_name='Jemima', last_name='Briones', age=3)